In [95]:
!pip install langchain pinecone-client transformers pypdf2 langchain_pinecone


In [103]:
from langchain.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore


In [104]:
llm = Ollama(model='llama3') 

In [105]:
def read_txt(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

# Read the text file
txt_text = read_txt("recipe.txt")  # Replace with your text file path


In [106]:
pdf_text

'RECIPE\nBOOKI am delighted to welcome you to our first Meals & More recipe book. At Brakes \nwe feel it is essential that children in the UK, whatever their circumstances have \naccess to good food 365 days of the year. From the start of our holiday hunger \nprogramme in 2015 we recognised that volunteers and professionals alike can give \na good service to their communities when they have the skills and resources to do \nso. That is why we have chosen the best of our most popular nutritionally balanced \nand easily prepared recipes for this book.\nWe have also been lucky enough to have wonderful contributions from some of our \nMeals & More industry chef supporters and we hope you enjoy their recipes.\nWe believe that everyone has a part to play in tackling child poverty and a great \nplace to start to truly make a difference is with preparing and sharing a healthy \nmeal.\nI hope you will not only enjoying cooking these great recipes at your community \nvenue but that you consider t

In [107]:
splitter = RecursiveCharacterTextSplitter(chunk_size=999, chunk_overlap=50)
chunks = splitter.split_text(pdf_text)

In [108]:
def embed_text(texts, embeddings_model):
    embeddings = embeddings_model.embed(texts)
    return embeddings

In [111]:
from langchain.embeddings import HuggingFaceEmbeddings


In [112]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [113]:
!pip install sentence-transformers

In [114]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [115]:
from pinecone import Pinecone as p1, ServerlessSpec
pc = p1(
    api_key='39c3b55b-2ae4-44ee-a9cd-83a99876c828'
)

In [116]:
index_name="recipe"
index = pc.Index("recipe")  
for i, t in zip(range(len(chunks)), chunks):
   query_result = embeddings_model.embed_query(t)
   print(i,t)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(t)} # meta data as dic
        }
    ],
    namespace="real" 
)

0 RECIPE
BOOKI am delighted to welcome you to our first Meals & More recipe book. At Brakes 
we feel it is essential that children in the UK, whatever their circumstances have 
access to good food 365 days of the year. From the start of our holiday hunger 
programme in 2015 we recognised that volunteers and professionals alike can give 
a good service to their communities when they have the skills and resources to do 
so. That is why we have chosen the best of our most popular nutritionally balanced 
and easily prepared recipes for this book.
We have also been lucky enough to have wonderful contributions from some of our 
Meals & More industry chef supporters and we hope you enjoy their recipes.
We believe that everyone has a part to play in tackling child poverty and a great 
place to start to truly make a difference is with preparing and sharing a healthy 
meal.
I hope you will not only enjoying cooking these great recipes at your community
1 venue but that you consider them part of 

In [117]:
from langchain_pinecone import PineconeVectorStore
import os
index_name = "recipe"
os.environ['PINECONE_API_KEY'] = '39c3b55b-2ae4-44ee-a9cd-83a99876c828'

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings_model,namespace="real")

In [118]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [119]:
retrieval_qa.run("give ingredients of Chili Mango Pasta Salad?")

In [ ]:
retrieval_qa.run("give ingredients of Pizza?")